#Importing libraries


In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
movie_data=pd.read_csv('/content/drive/MyDrive/Movie_Recommendation_System/movie_infob.csv',header=None)
cast_data=pd.read_csv('/content/drive/MyDrive/Movie_Recommendation_System/crew_infob.csv',header=None)
movie_id=pd.read_csv('/content/drive/MyDrive/Movie_Recommendation_System/movie_id_list.csv',header=[0])


In [3]:
#Combine both Dataframes
data=pd.concat([movie_id,movie_data,cast_data],axis=1,join='inner')

#Add Column Names
data.columns=['index','movie_id','movie_name','movie_genre','movie_overview','movie_cast','movie_director']


In [4]:
data.head()

,index,movie_id,movie_name,movie_genre,movie_overview,movie_cast,movie_director
0,0,4247,Scary Movie,Comedy,A familiar-looking group of teenagers find the...,"Anna Faris,Jon Abrahams,Marlon Wayans,Shawn W...",Keenen Ivory Wayans
1,1,11688,The Emperor's New Groove,"Adventure,Animation,Comedy,Family,Fantasy",Kuzco is a self-centered emperor who summons P...,"David Spade,John Goodman,Eartha Kitt,Patrick ...",Mark Dindal
2,2,10567,Dinosaur,"Family,Animation",An orphaned dinosaur raised by lemurs joins an...,"D. B. Sweeney,Alfre Woodard,Ossie Davis,Max C...",Ralph Zondag
3,3,8871,How the Grinch Stole Christmas,"Family,Comedy,Fantasy",Inside a snowflake exists the magical land of ...,"Jim Carrey,Taylor Momsen,Jeffrey Tambor,Chris...",Ron Howard
4,4,16366,Joseph: King of Dreams,"Family,Animation",In this animated retelling of the story from t...,"Ben Affleck,Mark Hamill,Richard Herd,Maureen ...",Rob LaDuca


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4400 entries, 0 to 4399
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           4400 non-null   int64 
 1   movie_id        4400 non-null   int64 
 2   movie_name      4400 non-null   object
 3   movie_genre     4387 non-null   object
 4   movie_overview  4391 non-null   object
 5   movie_cast      4400 non-null   object
 6   movie_director  4375 non-null   object
dtypes: int64(2), object(5)
memory usage: 240.8+ KB


#Handling null values

In [6]:
data.isnull().sum()

index              0
movie_id           0
movie_name         0
movie_genre       13
movie_overview     9
movie_cast         0
movie_director    25
dtype: int64

In [7]:
#Retrieving index which has null values
np.where(data['movie_genre'].isnull())[0]

array([  11,  676,  892, 1369, 1508, 1631, 1686, 1955, 2103, 2483, 3031,
       3738, 4040])

In [8]:
#retrieving movie_id from index
data.loc[11]

index                                                            11
movie_id                                                     118365
movie_name                                   Moses, Prince of Egypt
movie_genre                                                     NaN
movie_overview    At birth, Moses, a Hebrew baby is abandoned on...
movie_cast                                                         
movie_director                                  Ryszard Słapczyński
Name: 11, dtype: object

Performed API request for the Movie: "Moses,Prince of Egypt".





Movie_genre field was empty in the dictionary itself


In [9]:
#Taking a sample movie:Moses,Price of Egypt that has null value for movie_genre
import requests
API_key='55837420c664ac695063a44342aa40b1'

Movie_ID='118365'
url="https://api.themoviedb.org/3/movie/"+Movie_ID+"?api_key="+API_key+"&language=en-US"
response = requests.get(url).json()


In [10]:
response['genres']

[]

In [11]:

np.where(data['movie_overview'].isnull())[0]

array([ 892, 1369, 1987, 2740, 2851, 4344, 4352, 4360, 4385])

In [12]:
data.loc[892]

index                                                            92
movie_id                                                     687920
movie_name                                       Sexo e a Metrópole
movie_genre                                                     NaN
movie_overview                                                  NaN
movie_cast         Bárbara Paz,Fábio Hilst,Patrícia Coelho,Thays...
movie_director                                         Fred Avellar
Name: 892, dtype: object

In [13]:
#Taking a sample movie:Moses,Price of Egypt that has null value for movie_genre
import requests
API_key='55837420c664ac695063a44342aa40b1'

Movie_ID='687920'
url="https://api.themoviedb.org/3/movie/"+Movie_ID+"?api_key="+API_key+"&language=en-US"
response = requests.get(url).json()


In [14]:
response['overview']

''

Similarly, performed api request for few more movie_ids that had null value. Movie_genre fields were empty.

In [16]:
#retrieving movie_id from index
data.loc[777]

index                                                           177
movie_id                                                     693356
movie_name        One Piece Special: Open Upon the Great Sea! A ...
movie_genre                              Animation,Action,Adventure
movie_overview    Bonnie & Max are crew members of Pirate Zap's ...
movie_cast                                                         
movie_director                                                  NaN
Name: 777, dtype: object

Peformed API request. The movie_director field were empty as above for all the null values.

Also the movie_director fields fields were empty

In [20]:

#selecting  features
features=['movie_genre','movie_overview','movie_director']

In [21]:
for f in features:
  data[f] = data[f].fillna('')

In [22]:
data.isnull().sum()

index             0
movie_id          0
movie_name        0
movie_genre       0
movie_overview    0
movie_cast        0
movie_director    0
dtype: int64

#Importing libraries for data cleaning

In [ ]:

from rake_nltk import Rake
import nltk
nltk.download('stopwords')
nltk.download('punkt')

#converting movie_overview into key_words

In [25]:
#creating new dataframe
new_data=data[['movie_name','movie_genre','movie_overview','movie_cast','movie_director']]

In [ ]:
#Handling movie_overview

new_data['key_words']=""

for index,row in new_data.iterrows():
  overview=row['movie_overview']

  r=Rake()
  #retrieve keywords from overview data
  r.extract_keywords_from_text(overview)
  key_words_dict_scores=r.get_word_degrees()
  #append only the key to the list
  row['key_words']=list(key_words_dict_scores.keys())




In [ ]:
#Dropping movie_overview column
new_data.drop(columns=['movie_overview'],inplace=True)


In [28]:
new_data[['key_words']].head()

,key_words
0,"[familiar, looking, group, teenagers, find, st..."
1,"[kuzco, self, centered, emperor, summons, pach..."
2,"[orphaned, dinosaur, raised, lemurs, joins, ar..."
3,"[inside, snowflake, exists, magical, land, who..."
4,"[animated, retelling, story, bible, book, gene..."


#Handling movie_cast and director

In [30]:
new_data[['movie_cast','movie_director']].head()

,movie_cast,movie_director
0,"Anna Faris,Jon Abrahams,Marlon Wayans,Shawn W...",Keenen Ivory Wayans
1,"David Spade,John Goodman,Eartha Kitt,Patrick ...",Mark Dindal
2,"D. B. Sweeney,Alfre Woodard,Ossie Davis,Max C...",Ralph Zondag
3,"Jim Carrey,Taylor Momsen,Jeffrey Tambor,Chris...",Ron Howard
4,"Ben Affleck,Mark Hamill,Richard Herd,Maureen ...",Rob LaDuca


Merging all first and last names in one unique word.  

For example, **movie a** director is **Ben Affleck** and **movie B**  main actor is **Ben Kingsley** the model will detect a similarity because of their first name. We need to avoid that

In [ ]:
#At first, reemoving whitespaces between first and last name
#Then, removing commas between names

new_data['movie_cast']=new_data['movie_cast'].apply(lambda x:x.replace(" ","")).apply(lambda x:x.replace(","," "))

In [ ]:
#Removing whitesapces between first and last name of movie_directors

new_data['movie_director']=new_data['movie_director'].apply(lambda x:x.replace(" ",""))

In [33]:
new_data[['movie_cast','movie_director']].head()

,movie_cast,movie_director
0,AnnaFaris JonAbrahams MarlonWayans ShawnWayans...,KeenenIvoryWayans
1,DavidSpade JohnGoodman EarthaKitt PatrickWarbu...,MarkDindal
2,D.B.Sweeney AlfreWoodard OssieDavis MaxCasella...,RalphZondag
3,JimCarrey TaylorMomsen JeffreyTambor Christine...,RonHoward
4,BenAffleck MarkHamill RichardHerd MaureenMcGov...,RobLaDuca


#Handling movie_genre

In [ ]:
new_data['movie_genre']=new_data['movie_genre'].apply(lambda x:x.replace(","," "))


In [35]:
#convert "science fiction" to "sciencefiction" in movie_genre field
for i in new_data.index:
  if "science fiction" in new_data['movie_genre'][i]:
    #get the index value of the character 's' in 'science'
    a=new_data['movie_genre'][i].find("science fiction")
    #removing whitespace between the word science and fiction
    new_data['movie_genre'][i]=new_data['movie_genre'][i][:a+7]+new_data['movie_genre'][i][a+8:]
 

    
    

#Handling keywords


In [ ]:
#Converting list to string and removing commas between the key words
new_data['key_words']=new_data['key_words'].transform(lambda x:",".join(map(str,x))).apply(lambda x:x.replace(","," "))

In [37]:
new_data[['key_words']].head()

,key_words
0,familiar looking group teenagers find stalked ...
1,kuzco self centered emperor summons pacha vill...
2,orphaned dinosaur raised lemurs joins arduous ...
3,inside snowflake exists magical land whoville ...
4,animated retelling story bible book genesis jo...


#Model Creation

In [38]:
def create_soup(x):
    return x['key_words'] + ' ' +x['movie_cast']+ ' ' + x['movie_director'] + ' ' +x['movie_genre']

In [ ]:
new_data['soup'] = new_data.apply(create_soup, axis=1)


At first, the data was transformed into the vector array using **CountVectorizer** and found out that there are similar words
for example:
['abilities',
 'ability]

[accident',
 'accidental',
 'accidentally']

We need to use **Stemming**, in order to convert them to a single word or the root word

In [40]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [41]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [42]:
new_data['soup'][0]

'familiar looking group teenagers find stalked vaguely recognizable masked killer victims begin pile laughs none favorite scary movies escape razor sharp satire outrageously funny parody AnnaFaris JonAbrahams MarlonWayans ShawnWayans ReginaHall ShannonElizabeth CheriOteri LochlynMunro DaveSheridan KurtFuller CarmenElectra RickDucommun KeenenIvoryWayans MarissaJaretWinokur DanJoffre LloydBerry DavidL.Lander KellyCoffieldPark TrevorRoberts JayneTrcka KarenKruper TanjaReichert KendallSaunders GlynisDavies JessicaVanderVeen JenniferAnnLee FrankB.Moore GiacomoBaessato KyleGraham LeanneSantos MarkMcConchie MatthewPaxman ChrisRobson SusanShears PeterBryant AndreaNemeth CraigBruhnanski RegTupper BabeDolan DavidNeale NelsLennarson NicolaCrosbie IanBliss ChrisWilding PeterHanlon TedCole DoreenRamus LeeR.Mayes MarkHoeppner JimShepard DexterBell TedGill AndrewKeilty JamesVanDerBeek JohnnyHawkes KevinHansen ArtHives BruceMahler ZahfParoo AubreyTennant KeenenIvoryWayans Comedy'

In [43]:
stem('familiar looking group teenagers find stalked vaguely recognizable masked killer victims begin pile laughs none favorite scary movies escape razor sharp satire outrageously funny parody AnnaFaris JonAbrahams MarlonWayans ShawnWayans ReginaHall ShannonElizabeth CheriOteri LochlynMunro DaveSheridan KurtFuller CarmenElectra RickDucommun KeenenIvoryWayans MarissaJaretWinokur DanJoffre LloydBerry DavidL.Lander KellyCoffieldPark TrevorRoberts JayneTrcka KarenKruper TanjaReichert KendallSaunders GlynisDavies JessicaVanderVeen JenniferAnnLee FrankB.Moore GiacomoBaessato KyleGraham LeanneSantos MarkMcConchie MatthewPaxman ChrisRobson SusanShears PeterBryant AndreaNemeth CraigBruhnanski RegTupper BabeDolan DavidNeale NelsLennarson NicolaCrosbie IanBliss ChrisWilding PeterHanlon TedCole DoreenRamus LeeR.Mayes MarkHoeppner JimShepard DexterBell TedGill AndrewKeilty JamesVanDerBeek JohnnyHawkes KevinHansen ArtHives BruceMahler ZahfParoo AubreyTennant KeenenIvoryWayans Comedy')

'familiar look group teenag find stalk vagu recogniz mask killer victim begin pile laugh none favorit scari movi escap razor sharp satir outrag funni parodi annafari jonabraham marlonwayan shawnwayan reginahal shannonelizabeth cherioteri lochlynmunro davesheridan kurtful carmenelectra rickducommun keenenivorywayan marissajaretwinokur danjoffr lloydberri davidl.land kellycoffieldpark trevorrobert jaynetrcka karenkrup tanjareichert kendallsaund glynisdavi jessicavanderveen jenniferannle frankb.moor giacomobaessato kylegraham leannesanto markmcconchi matthewpaxman chrisrobson susanshear peterbry andreanemeth craigbruhnanski regtupp babedolan davidneal nelslennarson nicolacrosbi ianbliss chriswild peterhanlon tedcol doreenramu leer.may markhoeppn jimshepard dexterbel tedgil andrewkeilti jamesvanderbeek johnnyhawk kevinhansen arthiv brucemahl zahfparoo aubreytenn keenenivorywayan comedi'

In [ ]:
new_data['soup']=new_data['soup'].apply(stem)

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')


In [46]:
vectors = cv.fit_transform(new_data['soup']).toarray()

In [47]:
cv.get_feature_names()[:40]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['000',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '18th',
 '19',
 '1940',
 '1950',
 '1970',
 '1999',
 '19th',
 '20',
 '200',
 '30',
 '40',
 '50',
 '500',
 '50cent',
 '70',
 'aaron',
 'aaroneckhart',
 'aaronpaul',
 'aarontaylor',
 'aaronyoo',
 'aasifmandvi',
 'abandon',
 'abbiecornish',
 'abduct',
 'abigailbreslin',
 'abil',
 'abl',
 'abo',
 'aboard',
 'abrahambenrubi']

In [48]:
new_data['soup'].count()

4400

There are 4400 movies in total,which means we have 4400 vectors.
Each vectors contain 5000 features.

In the 5000 dimensional space each and every movie is plotted with their respective vectors

We need to calculate the distance between each movie vectors.
But in high dimensional space Euclidean distance fails.




Let us use Cosine similarity to calculate the angle between the each and every vectors

In [49]:
# Compute the Cosine Similarity 
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vectors)

In [50]:
#Each movie is compared with the rest of the movies and returns a similarity score
similarity.shape

(4400, 4400)

In [51]:
#Returns the similarity score of the First movie in the array with all the 4400 movies 
#the first value is 1 , as the movie is compared with itself
similarity[0]

array([1.        , 0.02626129, 0.        , ..., 0.03713907, 0.02680281,
       0.        ])

In [52]:
def get_recommendations(movie):
  #get the index of the movie
  movie_index=new_data[new_data['movie_name'] ==movie].index[0]
  distance=similarity[movie_index]
  #sort the list of movies based on the top 10 similarity scores keeping the index value intact using enumerate
  movies=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:11]

  for i in movies:
    #returns the movie_name
    print(new_data.iloc[i[0]].movie_name)
    

#get recommendation


In [53]:
get_recommendations('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman Unmasked: The Psychology of the Dark Knight
Shottas
Batman: Bad Blood
Batman: The Dark Knight Returns, Part 1
Brick Mansions
The Raid 2
Joker
Batman: Hush


In [54]:
get_recommendations('The Avengers')

Avengers: Age of Ultron
Avengers: Infinity War
Avengers: Endgame
Iron Man 2
Thor
Iron Man
Iron Man 3
Captain Marvel
Captain America: The Winter Soldier
Captain America: Civil War


In [55]:
import pickle

In [56]:
#inserting movie_id column
new_data.insert(loc=0,column='movie_id',value=data['movie_id'])

In [57]:
#saving the dataframe in the form of dictionary
pickle.dump(new_data.to_dict(),open('movies.pkl','wb'))

In [58]:
pickle.dump(similarity,open('similarity.pkl','wb'))